In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer
import pandas as pd
from sklearn.metrics import f1_score, classification_report
import os

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 커스텀 모델 로드
from model_training import MultiLabelClassifier

# 모델과 토크나이저 로드
model_path = r"D:\project\OSS_Project\AI\text-model\models\bert_model"
checkpoint = torch.load(os.path.join(model_path, "best_model.pt"))
model = MultiLabelClassifier()
model = model.to(device)  # GPU로 이동
model.load_state_dict(checkpoint['model_state_dict'])
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 메모리 효율을 위한 추가 코드 (OOM 방지)
import gc
torch.cuda.empty_cache()  # GPU 메모리 정리
gc.collect()  # 메모리 정리

# 테스트 데이터 로드
data_path = r"D:\project\OSS_Project\AI\text-model\data\preprocessed_data.csv"
data = pd.read_csv(data_path)

# 입력 문장 토크나이징
inputs = tokenizer(list(data['입력 문장']), padding=True, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}  # GPU로 이동

# 레이블 매핑 수정
도수_매핑 = {'낮은': 0, '중간': 1, '높은': 2, '알 수 없음': 3}
술종류_매핑 = {'칵테일': 0, '럼': 1, '위스키': 2, '보드카': 3, '알 수 없음': 4}
맛_매핑 = {'달달한': 0, '쓴맛': 1, '상큼한': 2, '신맛': 3, '부드러운': 4, '알 수 없음': 5}

# 레이블 변환
도수_labels = torch.tensor([도수_매핑[도수] for 도수 in data['도수']]).to(device)
술종류_labels = torch.tensor([술종류_매핑[종류] for 종류 in data['술 종류']]).to(device)
맛_labels = torch.tensor([맛_매핑[맛] for 맛 in data['맛']]).to(device)

# 데이터셋 생성
dataset = TensorDataset(
    inputs['input_ids'], 
    inputs['attention_mask'], 
    도수_labels,
    술종류_labels,
    맛_labels
)
BATCH_SIZE = 16
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

# 모델 평가
model.eval()
all_preds_도수 = []
all_preds_술종류 = []
all_preds_맛 = []
all_labels_도수 = []
all_labels_술종류 = []
all_labels_맛 = []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, 도수_label, 술종류_label, 맛_label = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # 각 분류기의 예측값 계산
        도수_pred = torch.argmax(outputs['도수'], dim=1)
        술종류_pred = torch.argmax(outputs['술종류'], dim=1)
        맛_pred = torch.argmax(outputs['맛'], dim=1)
        
        # 예측값과 실제값 저장 (CPU로 이동 후 저장)
        all_preds_도수.extend(도수_pred.cpu().numpy())
        all_preds_술종류.extend(술종류_pred.cpu().numpy())
        all_preds_맛.extend(맛_pred.cpu().numpy())
        all_labels_도수.extend(도수_label.cpu().numpy())
        all_labels_술종류.extend(술종류_label.cpu().numpy())
        all_labels_맛.extend(맛_label.cpu().numpy())

# 각 분류기별 성능 평가
print("도수 분류 결과:")
print(classification_report(all_labels_도수, all_preds_도수, 
                          target_names=list(도수_매핑.keys())))

print("\n술종류 분류 결과:")
print(classification_report(all_labels_술종류, all_preds_술종류, 
                          target_names=list(술종류_매핑.keys())))

print("\n맛 분류 결과:")
print(classification_report(all_labels_맛, all_preds_맛, 
                          target_names=list(맛_매핑.keys())))

Using device: cuda


C:\Users\user\AppData\Local\Temp\ipykernel_25476\2974539093.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(model_path, "best_model

도수 분류 결과:
              precision    recall  f1-score   support

          낮은       1.00      0.92      0.96       427
          중간       1.00      0.94      0.97       476
          높은       0.88      0.93      0.91       420
      알 수 없음       0.64      0.80      0.71       164

    accuracy                           0.92      1487
   macro avg       0.88      0.90      0.89      1487
weighted avg       0.93      0.92      0.92      1487


술종류 분류 결과:
              precision    recall  f1-score   support

         칵테일       1.00      0.91      0.95       357
           럼       1.00      0.93      0.96       333
         위스키       1.00      0.91      0.95       318
         보드카       1.00      0.91      0.95       314
      알 수 없음       0.60      1.00      0.75       165

    accuracy                           0.93      1487
   macro avg       0.92      0.93      0.92      1487
weighted avg       0.96      0.93      0.93      1487


맛 분류 결과:
              precision    recall  f1-score 